<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Veracity60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import fastai

In [0]:
from fastai.text import * 

In [0]:
def read(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 3.0:
            return 'Query'
        elif col['label'] == 2.0:
            return 'Deny'
        elif col['label'] == 1.0:
            return 'Support'
        elif col['label'] == 0.0:
            return 'Comment'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [0]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [0]:
# For veracity
train = read_veracity('train.csv', True)
test = read_veracity('test.csv')

In [0]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [7]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [8]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [11]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.058336,4.690670,0.266518,00:00


In [12]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.753201,3.923149,0.326786,00:00


In [13]:
learn.predict("This is a review about", n_words=10)

'This is a review about the people in the U.S. , which is the'

In [0]:
learn.save_encoder('ft_enc')

In [15]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (217 items)
x: TextList
xxbos xxmaj xxunk for # xxmaj ferguson police chief ? xxmaj if xxup officer xxup darren xxup wilson was in xxunk of a robbery suspect , xxunk tell xxunk to get out of the road ? xxup lie,xxbos xxmaj two shot in xxmaj paris , including xxunk cop , as manhunt xxunk for # charliehebdo suspects http : / / t.co / xxunk http : / / t.co / xxunk,xxbos xxup xxunk xxmaj police confirm 5 hostages have xxunk from # sydneysiege and they are xxunk : " xxmaj it xxunk take a xxunk of time " http : / / t.co / xxunk,xxbos xxmaj canadian authorities have xxunk name of suspect in xxmaj ottawa attacks to xxup u.s. xxunk ; xxunk for xxup fbi xxunk : xxmaj xxunk xxup u.s. law xxunk official,xxbos xxmaj xxunk reports xxunk that xxmaj michael xxmaj xxunk has xxunk xxmaj xxunk . http : / / t.co / xxunk
y: CategoryList
Unverified,True,True,Unverified,False
Path: .;

Valid: LabelList (55 items)
x: TextList
xxbos xxunk xxunk xxunk - at - 

In [16]:
data_clas.show_batch()

text,target
"xxbos xxup developing : xxmaj gunman xxmaj xxunk xxmaj hostages xxmaj in xxmaj sydney xxmaj cafe , ' xxup isis ' xxmaj flags xxmaj being xxmaj held xxmaj against xxmaj window - http : / / t.co / xxunk http : / / t.co / xxunk",True
"xxbos xxmaj name of # xxmaj ferguson cop is xxunk to be released by xxunk xxup et , police chief says : http : / / t.co / xxunk ( xxup xxunk xxmaj xxunk , xxup ap ) http : / / t.co / xxunk",Unverified
"xxbos xxmaj xxunk like xxunk xxmaj charlie xxmaj hebdo xxunk has been killed : 1 . xxmaj charb - was on xxmaj al - xxmaj xxunk 's xxunk xxunk ; 2 . xxmaj cabu , 3 . xxmaj wolinski , 4 . xxmaj tignous",Unverified
xxbos xxmaj flag in window of xxmaj sydney xxmaj lindt cafe not an xxup isis flag . xxmaj reads : ‘ xxmaj there is no xxmaj god but xxmaj xxunk and xxmaj muhammad is the xxunk of xxmaj god ’ ( xxunk ),True
"xxbos xxmaj xxunk xxunk xxunk were xxunk last xxunk -- xxunk all by other xxunk . a xxunk xxunk were unarmed - killed - by - cop . \n xxmaj where 's xxmaj al , xxmaj xxunk ? \n # xxunk # ferguson",Unverified


In [17]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,1.203685,0.989854,0.454545,00:00


In [18]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.982516,0.842645,0.654545,00:00


In [19]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.804797,0.832272,0.672727,00:00
1,0.789277,0.808669,0.654545,00:00
2,0.751655,0.814258,0.600000,00:00
3,0.694939,0.731514,0.690909,00:00
4,0.637324,0.775593,0.690909,00:00
5,0.574417,0.749058,0.690909,00:00
6,0.517025,0.681740,0.709091,00:00
7,0.458799,0.681962,0.745455,00:00
8,0.414714,0.697944,0.745455,00:00
9,0.368310,0.694207,0.709091,00:00


In [0]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [0]:
from sklearn.metrics import accuracy_score, f1_score

In [22]:
accuracy_score(test['label'], l)

0.6071428571428571

In [23]:
f1_score(test['label'], l, average='macro')

0.6066047471620227

In [0]:
learn.export('veracity.pkl')